In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle

from keras.models import Sequential, Model
from keras.layers import Conv2D
import os
from keras.applications.vgg16 import VGG16


In [2]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 1024 #Resize images (height  = X, width = Y)
SIZE_Y = 996

In [3]:
#Capture training image info as a list
train_images = []

In [5]:
for directory_path in glob.glob("/home/dsp_tameo/tamiyo-p3/data/train/data/007f736aedbc4ca67989f8ca62f1bbeb447ad76698351fe387923963ee50e5ae"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        #train_labels.append(label)

In [6]:
train_images

[array([[[116, 116, 116],
         [116, 116, 116],
         [115, 115, 115],
         ...,
         [ 87,  87,  87],
         [ 88,  88,  88],
         [ 88,  88,  88]],
 
        [[116, 116, 116],
         [116, 116, 116],
         [115, 115, 115],
         ...,
         [ 87,  87,  87],
         [ 88,  88,  88],
         [ 88,  88,  88]],
 
        [[116, 116, 116],
         [116, 116, 116],
         [115, 115, 115],
         ...,
         [ 87,  87,  87],
         [ 88,  88,  88],
         [ 88,  88,  88]],
 
        ...,
 
        [[100, 100, 100],
         [100, 100, 100],
         [ 99,  99,  99],
         ...,
         [104, 104, 104],
         [104, 104, 104],
         [104, 104, 104]],
 
        [[100, 100, 100],
         [100, 100, 100],
         [ 99,  99,  99],
         ...,
         [104, 104, 104],
         [104, 104, 104],
         [104, 104, 104]],
 
        [[100, 100, 100],
         [100, 100, 100],
         [ 99,  99,  99],
         ...,
         [104, 104, 104],
  